---

---

---

# Chapter 2: Modular Code Design for SimPy Simulations

---

### Overview

In this chapter, we will explore the reusable, modular code structures I developed to implement our homework projects using SimPy.  
These building blocks enable flexible and efficient simulation modeling across different scenarios.

---

---

---

## 🔢 Number Generators: Creating Random Variables

These classes generate random numbers for inter-arrival and service times in the simulation.

```python
class Exponential_Generator:
    def __init__(self, seed:int, mean:float) -> None:
        self.mean = mean
        self.rng = np.random.default_rng(seed=seed)

    def __call__(self):
        return self.rng.exponential(self.mean)
```

* This class uses NumPy’s random generator to produce exponential random variables.
* We call it like a function to get a new sample every time (e.g., `arrival_gen()`).


## 🧍 Customer: The Basic Unit of the Simulation

Each customer is a process that requests the server, waits, gets served, and leaves.

```python
def process(self):
    with self.server.request() as request:
        yield request  # Wait for the server to be free

        wait_time_in_queue = self.env.now - self.arrival_time
        yield self.env.timeout(self.service_time)  # Service time

        wait_time_in_system = wait_time_in_queue + self.service_time
        # Update statistics here
```

* The customer **requests** access to the server (resource).
* Waits if busy, then uses the server for `service_time`.
* Tracks and saves waiting times for analysis.



## ⚙️ Queue: The Simulation Engine

Manages arrivals, queue, and dispatching customers to servers.

### Key parts of the queue:

---

1. **Arrival Process:** Generates customers according to arrival distribution.

```python
while True:
    interarrival = self.arrival_gen()
    yield self.env.timeout(interarrival)  # Wait between arrivals

    # Create new customer and add to queue
```

---

2. **Dispatcher Process:** Sends waiting customers to the server as soon as it is free.

```python
while True:
    # Wait for event of customer arrival or completion
    result_event = yield self.callbacks["customer_arrived_event"] | self.callbacks["dispatcher_event"]

    if len(self.server.users) < self.server.capacity and len(self.customers_in_queue) > 0:
        customer_id = policy_function(self.customers_in_queue)
        self.customer_in_system[customer_id].active()  # Start service
```

---

* The **simulation environment** (`self.env`) advances step by step.
* The queue can be stopped by time or by a customer limit.
* The modular design lets us easily change arrival/service generators or dispatch policies.
